In [7]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


,Count
0,224


Build a query to fetch all the parks in Utah (UT) and order the results by the park name.

In [2]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_has_any(states_list, ['UT'])
ORDER by fullname asc;

-- done

,fullName,states_list


In [74]:
%%sql

SELECT
    name,
    states
from nps_public_data.parks p
where p.states like '%UT%'
ORDER by 1
LIMIT 5;

,name,states
0,Arches,UT
1,Bryce Canyon,UT
2,California,"CA,CO,ID,KS,MO,NE,NV,OR,UT,WY"
3,Canyonlands,UT
4,Capitol Reef,UT


Build a query to fetch all the National Parks that cross state boundaries (i.e. parks that are in more than one state). Remember to filter `parks` for *only* National Parks.

Hint: `parks.states` is a string representation of a list, i.e. `UT,CA,NC`. The `parks` table includes parks that aren't National Parks.

In [79]:
%%sql


    SELECT 
        name, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where p.designation ILIKE '%National Park%' and states_count > 1
    order by 1;


,name,designation,states_list,states_count
0,Death Valley,National Park,"[CA, NV]",2
1,Great Smoky Mountains,National Park,"[NC, TN]",2
2,Yellowstone,National Park,"[ID, MT, WY]",3


In [2]:
%%sql

SELECT
    name,
    states
from nps_public_data.parks p
where p.states like '%,%'
and p.designation ILIKE '%National Park%'
ORDER by 1


,name,states
0,Death Valley,"CA,NV"
1,Great Smoky Mountains,"NC,TN"
2,Yellowstone,"ID,MT,WY"


For all national parks, return the `states` column as a `STRUCT` type with each element as a state.

In [3]:
%%sql


    SELECT
        name,
        split(states, ',')::STRING[] AS states_list
    FROM nps_public_data.parks
    WHERE designation ILIKE '%National Park%'



,name,states_list
0,Isle Royale,[MI]
1,Black Canyon Of The Gunnison,[CO]
2,Grand Canyon,[AZ]
3,Crater Lake,[OR]
4,Yellowstone,"[ID, MT, WY]"
...,...,...
56,Voyageurs,[MN]
57,Mesa Verde,[CO]
58,Petrified Forest,[AZ]
59,Virgin Islands,[VI]


Which parks are in **either** Montana (MT) or Wyoming (WY)?

In [83]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_has_any(states_list, ['MT', 'WY'])
and designation ILIKE '%National Park%'
ORDER by fullname asc;


,fullName,states_list
0,Glacier National Park,[MT]
1,Grand Teton National Park,[WY]
2,Yellowstone National Park,"[ID, MT, WY]"


What about _both_ Montana _and_ Wyoming?

In [85]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_has_all(states_list, ['MT', 'WY']) and designation ILIKE '%National Park%'
ORDER by fullname asc;

,fullName,states_list
0,Yellowstone National Park,"[ID, MT, WY]"


In [84]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_contains(states_list, 'MT') and list_contains(states_list, 'WY') and designation ILIKE '%National Park%'
ORDER by fullname asc;

,fullName,states_list
0,Yellowstone National Park,"[ID, MT, WY]"


Which park is in the greatest number of states? How many states>

In [4]:
%%sql

    SELECT 
        fullname, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where designation ILIKE '%National Park%'
    order by states_count desc, fullname asc
    limit 5;




,fullName,designation,states_list,states_count
0,Yellowstone National Park,National Park,"[ID, MT, WY]",3
1,Death Valley National Park,National Park,"[CA, NV]",2
2,Great Smoky Mountains National Park,National Park,"[NC, TN]",2
3,Acadia National Park,National Park,[ME],1
4,Arches National Park,National Park,[UT],1


Now, how many parks are in each "group" of state border-crossings? i.e. how many parks are in 1 state, 2 states, 3 states, etc.

Hint: we're grouping by the _number_ of states.

In [9]:
%%sql

with states_list as (
    SELECT 
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where designation ILIKE '%National Park%'
)

select 
  states_count,
  count(*) as parks_count
from states_list
group by states_count
order by states_count 

    
  


,states_count,parks_count
0,1,58
1,2,2
2,3,1


What's the percentage share of the total? i.e. what *percentage* of parks are in 1 state, 2 states, etc

Hint: window functions might be helpful.

In [17]:
%%sql

with states_list as (
    SELECT 
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where designation ILIKE '%National Park%'
)

select 
  states_count,
  count(*) as parks_count,
  sum(count(*)) over() as total_states
from states_list
group by states_count
order by states_count 


,states_count,parks_count,total_states
0,1,58,61.0
1,2,2,61.0
2,3,1,61.0


Write a query that returns the _largest_ campsite in each park. 

As a bonus, do this _without_ using a CTE (filter in the same query)

In [22]:
%%sql

SELECT 
    p.name as park_name,
    c.name as campground_name,
    ROW_NUMBER() OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as park_campsites_rank,
    c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as num_campsites

FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode and p.designation ILIKE '%National Park%'
QUALIFY park_campsites_rank = 1
order by num_campsites desc, park_name


,park_name,campground_name,park_campsites_rank,num_campsites
0,Mesa Verde,Morefield Campground,1,534
1,Yellowstone,Bridge Bay Campground,1,432
2,Grand Teton,Colter Bay Campground,1,347
3,Grand Canyon,Mather Campground - South Rim,1,315
4,Yosemite,Tuolumne Meadows Campground,1,304
5,Acadia,Blackwoods Campground,1,281
6,Rocky Mountain,Moraine Park Campground,1,239
7,Death Valley,Sunset Campground,1,230
8,Shenandoah,Big Meadows Campground,1,220
9,Everglades,Long Pine Key Campground,1,216


In [59]:
%%sql

-- List all tables with schema information
SELECT * FROM information_schema.tables WHERE table_schema NOT IN ('information_schema', 'pg_catalog') LIMIT 2;

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,nps_public_data,activities,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None,None
1,memory,nps_public_data,activities__parks,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None,None


Say you'll be in California (CA) this spring and have time for three National Parks visits. 

How many combinations of national parks can you visit? Can you return the combinations in a list ordered by the name of the first park?

Hint: self-joins and cross joins might be useful.

In [23]:
%%sql

with california_parks as (
    SELECT 
        name, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where p.designation ILIKE '%National Park%' and list_contains(states_list, 'CA')
)

select c1.name as park1_name,
       c2.name as park2_name,
       c3.name as park3_name

from california_parks as c1
cross join california_parks as c2
cross join california_parks as c3
where c1.name != c3.name and c2.name != c3.name and c1.name != c2.name

,park1_name,park2_name,park3_name
0,Joshua Tree,Channel Islands,Death Valley
1,Lassen Volcanic,Channel Islands,Death Valley
2,Sequoia & Kings Canyon,Channel Islands,Death Valley
3,Pinnacles,Channel Islands,Death Valley
4,Yosemite,Channel Islands,Death Valley
...,...,...,...
205,Channel Islands,Yosemite,Pinnacles
206,Death Valley,Yosemite,Pinnacles
207,Joshua Tree,Yosemite,Pinnacles
208,Lassen Volcanic,Yosemite,Pinnacles


For our final exercise— find the combinations in alphabetical order, that is, the first letter of each visit occurs in the order of the alphabet, e.g. `[C]hannel Islands, [D]eath Valley, [J]oshua Tree` would satisfy that condition.

Hint: how can you compare strings for alphabetical order? (casing matters)

In [71]:
%%sql

with california_parks as (
    SELECT 
        name, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where p.designation ILIKE '%National Park%' and list_contains(states_list, 'CA')
)

select c1.name as park1_name,
       c2.name as park2_name,
       c3.name as park3_name

from california_parks as c1
cross join california_parks as c2
cross join california_parks as c3
where c1.name != c3.name and c2.name != c3.name and c1.name != c2.name
and c1.name < c2.name and c1.name < c3.name and c2.name < c3.name
order by c1.name, c2.name, c3.name


,park1_name,park2_name,park3_name
0,Channel Islands,Death Valley,Joshua Tree
1,Channel Islands,Death Valley,Lassen Volcanic
2,Channel Islands,Death Valley,Pinnacles
3,Channel Islands,Death Valley,Sequoia & Kings Canyon
4,Channel Islands,Death Valley,Yosemite
5,Channel Islands,Joshua Tree,Lassen Volcanic
6,Channel Islands,Joshua Tree,Pinnacles
7,Channel Islands,Joshua Tree,Sequoia & Kings Canyon
8,Channel Islands,Joshua Tree,Yosemite
9,Channel Islands,Lassen Volcanic,Pinnacles
